In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Step 1: Get the top 50 classical chess players
top_players_url = "https://lichess.org/player/top/50/classical"
response = requests.get(top_players_url)



if response.status_code == 200:
    top_players_data = response.json()
    print(top_player_data)
    top_players_usernames = [player["username"] for player in top_players_data["users"]]
else:
    print("Error getting top players:", response.status_code)
    exit()

# Step 2: Get the rating history for the top player for the last 30 days
top_player_username = top_players_usernames[0]  # Assuming the top player is at index 0
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

rating_history_url = f"https://lichess.org/api/user/{top_player_username}/rating-history?perfType=classical&rated=true&since={start_date.timestamp()}&until={end_date.timestamp()}"
response = requests.get(rating_history_url)

if response.status_code == 200:
    rating_history_data = response.json()
    rating_history = rating_history_data[top_player_username]["points"]
    rating_dates = [datetime.fromtimestamp(item["date"]) for item in rating_history]
else:
    print("Error getting rating history:", response.status_code)
    exit()

# Step 3: Create a CSV file with the required information
data = {"Username": [top_player_username] * 31,
        "Rating 30 Days Ago": [rating_history[0]["rating"]] * 31,
        "Rating Today": [rating_history[-1]["rating"]] * 31}

for i in range(len(rating_history)):
    data[f"Rating {i} Days Ago"] = [rating_history[i]["rating"]] * 31

df = pd.DataFrame(data)
df.to_csv("chess_ratings.csv", index=False)
print("CSV file created successfully.")


Error getting top players: 404


NameError: name 'top_players_usernames' is not defined